# Self Query Retriever Test Notebook

## Remove old database

In [1]:
import os
import shutil
from loguru import logger

vs_path = "test_data/qdrant_vector_store/test_vct_store"
vsc_path = os.path.join(vs_path, "collections")
collection_name = "Steve Jobs' 2005 Stanford Commencement Address"

for path in [vsc_path, vs_path]:
    if os.path.exists(path):
        logger.info(f"Removing path: {path}")
        shutil.rmtree(path)

## Initialize embedding model

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model_name: str = 'sentence-transformers/all-MiniLM-L12-v2'
embedding_model = HuggingFaceEmbeddings(model=embedding_model_name)

## Create the vector store

In [3]:
from intellitube.vector_store import VectorStoreManager

vdb = VectorStoreManager(
    embedding_model=embedding_model,
    path_on_disk=vs_path,
    collection_path_on_disk=vsc_path,
    collection_name=collection_name,
)

client = vdb.client
vectorstore = vdb.vectorstore

2025-07-13 23:36:12.029 | DEBUG    | intellitube.vector_store:init_vector_store:120 - New Qdrant Client Initialized.
2025-07-13 23:36:12.038 | DEBUG    | intellitube.vector_store:init_vector_store:126 - New Vector Store Created.


## Loading YouTube Video Transcript

In [4]:
from intellitube.utils import (
    YTContentData, download_youtube_audio_or_transcript, webvtt_2_str
)

video_url = "https://www.youtube.com/watch?v=UF8uR6Z6KLc"
video_data: YTContentData = download_youtube_audio_or_transcript(video_url)
vtt_str = webvtt_2_str(vtt_file_path=video_data.transcript_path)

print(video_data, end='\n\n')
print(vtt_str[:500])

2025-07-13 23:36:12.154 | DEBUG    | intellitube.utils.youtube:download_youtube_audio_or_transcript:133 - Cache exists, validating cache...
2025-07-13 23:36:12.154 | DEBUG    | intellitube.utils.youtube:download_youtube_audio_or_transcript:145 - Cache contains the requested data. Using cache.


type='text' transcript_path='test_data/cache/youtube/downloads/0290de54-13f5-4bbe-b3de-7b06f46a5f07.vtt' audio_path=None

This program is brought to you by Stanford University.
Please visit us at stanford.edu
Thank You. I am honored to be with you today at your commencement
from one of the finest universities in the world.
Truth be told I never graduated from college
and this is the closest I've ever gotten to a college graduation.
Today I want to tell you three stories from my life. That's it.
No big deal. Just three stories.
The first story is about connecting the dots.
I dropped out of Reed College after the fir


## Split texts & Add them to VDB

### 1. Split Texts + Create Metadatas

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=256,
)

texts = text_splitter.split_text(vtt_str)
metadatas = [
    {"source": video_url, "chunk_index": i + 1}
    for i in range(0, len(texts))
]

### 2. Add them to VDB

In [6]:
vectorstore.add_texts(texts=texts, metadatas=metadatas)

['2f522f07e13c44bba4a0a547f796cbef',
 '0f885dd85fb948d6bb6252aef3f165a2',
 '30cb9fd1172b460ab62c1770fecabcc3',
 '220c7be2e5fc427d95c39cb6dfd0dfcb',
 '8645a6db70b64f9f905215285e449d90',
 '62b1e7ef3bdc4121ad7d0c9602fcf46f',
 '53dd2e8d6afb4aad8eb4095f1f21c3d9',
 '0dadd656405d48e089567741c52ed202',
 '7bfd3f5be7af4edb90b99d80065d8407',
 '10d1e7bd1ef74787a949c28c0eb8c9c5',
 'e5f21689d96044c3a48c70a2a9a671c9',
 '7c5b7bb8692243328eb7df91644350aa',
 'e127ce71e0744f74bcbeb68fc7af378d',
 '917bd957796448b789fb2bba9785f860',
 '022045a60f2147ee9017c31542d033e6',
 'd5f4d67cd9c849cc91906596065f5e4f',
 'd14fda8f46e54fd4a4ef6f11a9fce90d',
 '3c42c85cf72042779281217ed25dd77c',
 'bd3596b1c07c4d18b343c8e937d52051',
 '881fe084a2c941a2ad196127459a1741',
 'e56eb2c3f9ec4d53973766f1b2d0584a',
 'ced44b2ba1d94964a317216a4b9d535a',
 'efe085c42166492eb4f90b0ed71bbef5',
 '9c24b2c0541840cfb163a13950405485',
 '689c40e91c354cb9a73d6148190674de',
 '042eda2fe88045c5aa2fd54b40574874',
 'b4b092bb1ece47c1865e815e18fa4c2d',
 

## Create the Retriever

In [7]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.6}
)

## Retrieve all the documents by metadata filtering
Thanks to this [stackoverflow question!](https://stackoverflow.com/questions/78118020/qdrant-client-scroll-filter-does-not-work)

In [8]:
from qdrant_client import models

scroll_result = client.scroll(
    collection_name=collection_name,
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="metadata.source", match=models.MatchValue(value=video_url)),
            # models.FieldCondition(key="metadata.chunk_index", match=models.MatchValue(value=2)),
        ]
    ),
    limit=1000,
    with_payload=True,
    with_vectors=False
)

print(len(scroll_result[0]))
# print(scroll_result[1])

47


In [9]:
vectorstore.similarity_search("why is death beautiful?", k=5)

[Document(metadata={'source': 'https://www.youtube.com/watch?v=UF8uR6Z6KLc', 'chunk_index': 40, '_id': '0ae722bda7ad4942b07580dd5824411d', '_collection_name': "Steve Jobs' 2005 Stanford Commencement Address"}, page_content="I can now say this to you with a bit more certainty than when\ndeath was a useful but purely intellectual concept:\nNo one wants to die.\nEven people who want to go to heaven don't want to die to get there.\nAnd yet death is the destination we all share.\nNo one has ever escaped it. And that is as it should be,\nbecause Death is very likely the single best invention of Life.\nIt is Life's change agent.\nIt clears out the old to make way for the new.\nRight now the new is you, but someday not too long from now,"),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=UF8uR6Z6KLc', 'chunk_index': 33, '_id': '9bf958ef176c4823a6120c3c67c835c3', '_collection_name': "Steve Jobs' 2005 Stanford Commencement Address"}, page_content="Remembering that I'll be dead soo